# MIBIsualization GUI

Notebook for easily viewing different mass channels in a single MIBItiff  
Future updates will add support for multiple tiffs (maybe drop down file selection?)


In [1]:
%matplotlib widget
#%config InlineBackend.figure_format = 'svg'

from ipywidgets import Layout, GridBox, Button, ButtonStyle, GridspecLayout, FloatSlider, FloatLogSlider, Output, Select, HBox, VBox, HTML, Dropdown, Text

import io
import PIL.Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import pathlib
import glob

from mibidata import mibi_image as mi, tiff
import visualize_data as viz

from traitlets import HasTraits, observe

In /usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The pgf.debug rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
In /usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In /usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The verbose.fi

In [2]:
%%html
<style>
div.jupyter-widgets.widget-label {display: none;}
</style>

## Image View Screen

defining functions for controlling image toggling

In [3]:

# globals :/
loaded_ims = []
pltimage = None

# this nasty thing is global (i should honestly just make all this object-orienty)
idx = 0
# this nasty thing is also global
options = []

# output widget
im_screen = Output(layout=Layout(width='1200px', height='1200px'))

# plotting parameters (easier to set as defaults, rather than update each time)
plt.rcParams["figure.figsize"] = (1024/96,1024/96)
plt.rcParams["figure.dpi"] = 96

# axes styling parameters
def style_plot():
    plt.gca().set_frame_on(False)
    plt.gca().xaxis.set_visible(False)
    plt.gca().yaxis.set_visible(False)
    plt.tight_layout(pad=0)

# plot empty image
def empty_image():
    empty = np.zeros((1024,1024))
    
    global idx
    idx = 0
    
    pltimage.set_data(empty)

# apply contrast filter
def apply_contrast_filter(index, gp, mag):
    fim = np.copy(loaded_ims[index])
    #print(np.amin(fim))
    #print(gp)
    #print(mag)
    if gp > 0.:
        fim[fim < gp] = np.power(fim[fim < gp],mag)/gp**(mag-1)
    if gp < 1.:
        fim[fim > gp] = 1 - np.power(1-fim[fim>gp],mag)/(1-gp)**(mag-1)
    return fim

# greypoint slider
b_greypoint = FloatSlider(description='g',
                          value=0,
                          min=0,
                          max=1.0,
                          step=0.05,
                          orientation='vertical',
                          layout=Layout(display='flex', flex='flex-shrink',width='auto',height='auto'),
                          readout = True,
                          readout_format = '.2f',
                          continuous_update = False
                         )

# callback for changing the contrast magnitude
def change_contrast_mag(change):
    mag = change['new']
    pltimage.set_data(apply_contrast_filter(idx,b_greypoint.value,mag))

# contrast magnitude slider
b_cef = FloatLogSlider(description='c',
                    value=1.0,
                    base=10,
                    min=-1,
                    max=1,
                    step=0.1,
                    orientation='vertical',
                    layout=Layout(display='flex', flex='flex-shrink',width='auto',height='auto'),
                    readout=True,
                    readout_format = '.2f',
                    continuous_update = False
                   )

# callback for changing the contrast greypoint
def change_contrast_gp(change):
    gp = change['new']
    pltimage.set_data(apply_contrast_filter(idx,gp,b_cef.value))

# contrast-callback assignment
b_greypoint.observe(change_contrast_gp, names='value')
b_cef.observe(change_contrast_mag, names='value')

# plot at idx
def change_image(change):
    
    global idx
    idx = change['new']
    
    if not len(loaded_ims) > 0:
        return empty_image()
    
    pltimage.set_data(apply_contrast_filter(idx,b_greypoint.value,b_cef.value))

## File Selection Mode

window mode for managing image uploads (adding or removing images or channels)

In [4]:
file_select = GridspecLayout(n_rows=12, 
                             n_columns=12,
                             height='1024px',
                             width='1024px'
                            )
file_select.layout.border = border='solid 2px'

fs_panel_label = HTML(value='Panel CSV: ')

fs_panel_value = HTML(value='<font color="#aaaaaa"><i>Please load a panel...</i></font>')




fs_dropdown = Dropdown(description='Folder', layout=Layout(width='auto'))

OShome = pathlib.Path.home()
quick_flag = False
def parent_crawl(home):
    p = home
    yield p
    while p != pathlib.Path('/'):
        p = p.parent
        yield p

def assign_home(change):
    global quick_flag
    if change['old'] != change['new'] and not quick_flag:
        quick_flag = True
        home = pathlib.Path(change['new'])
        fs_dropdown.options = [str(p) for p in parent_crawl(home)][::-1]
        fs_dropdown.value = str(home)
        quick_flag = False
    elif quick_flag:
        quick_flag = False
        
assign_home({'new' : OShome, 'old' : pathlib.Path('/')})

fs_dropdown.observe(assign_home, names='value')


fs_top_panel = HBox(children=[fs_panel_label, 
                              fs_panel_value,
                              fs_dropdown
                             ], 
                    layout=Layout(display='inline-flex', 
                                  flex_flow='row',
                                  justify_content='space-around',
                                  align_content='flex-start',
                                  width='auto', 
                                  height='auto'
                                 )
                   )





file_select[0:1,:] = fs_top_panel

# panel display/selector
file_select[1:,0:3] = Button(layout=Layout(width='auto',height='auto'),style=ButtonStyle(button_color='darkseagreen'))

# file nav 1
file_select[1:10,3:7] = Button(layout=Layout(width='auto',height='auto'),style=ButtonStyle(button_color='darkseagreen'))

# subfile nav
file_select[1:10,7:] = Button(layout=Layout(width='auto',height='auto'),style=ButtonStyle(button_color='darkseagreen'))

# buttons for adding selection to viewer and closing window
file_select[10:,3:12] = Button(layout=Layout(width='auto',height='auto'),style=ButtonStyle(button_color='darkseagreen'))


#file_select = Button(layout=Layout(width='1024px',height='1024px'),style=ButtonStyle(button_color='darkseagreen'))

/Users/adam_kagel


## Button Panel

important buttons:
 - contrast enhancement (greypoint + magnitude)  'g' + 'c'
 - add image (just mass channels for now) 'a'
 - remove image (" " " " ") 'r'
 - move image relatively up 'u'
 - move image relatively down 'd'

In [5]:
# add image to list
#b_add = Button(description='a', 
#               layout=Layout(width='auto')
#              )
#
# remove image from list
#b_rem = Button(description='r',
#               layout=Layout(width='auto')
#              )
#
# would an upload manager mode work better than an add+remove button?  Yes, yes it would

# switch to upload managing mode
b_manage = Button(description='m',
                  layout=Layout(width='auto')
                  )

# move image up in list
b_up = Button(description='u',
              layout=Layout(width='auto')
             )

# move image down in list
b_down = Button(description='d',
                layout=Layout(width='auto')
               )

# bundle controls into gui panel
buttons = HBox(
            children=[b_greypoint,
                      b_cef,
                      b_manage,
                      b_up,
                      b_down
                     ],
            layout=Layout(display='flex',
                          flex_flow='row',
                          justify_content='space-around',
                          align_content='flex-start',
                          width='auto',
                          height='auto'
                         )
            )

## File List

Shows list of loaded images

 - For testing purposes, this is just loading and displaying three togglable images

In [6]:
## FOR TESTING LIST CAPABILITY ##

## Normally, adding images will be handled dynamically in the button panel ##

data_path = pathlib.Path('../sample_data/')
#save_path = pathlib.Path('../svdOUT/')

mass_channels = [89,115,197]

# background channel to compare to.  For no background comparison, set to -1
# bg_channel = 197

# Option for comparing to bg channel
# pixels w/ non zero values -> 1.0 (shows all overlaps equally)
# binary = False

# Anonymized by default currently
# TODO: Read panel if anonymize is false, and use target names
#anonymize_targets = True
#panel_path = data_path.joinpath('Panel54_MelanomaCohort_2.csv')

# slide foldernames for input folders
slide_foldernames = [
    'CohortSlide1'
]

# slide designations for output folders
# leave empty to use full slide name
slide_names = [
    'S1'
]

process_folder_prefix = ''
process_folder_suffix = '_TIFF'

raw_tiff_folder_name = 'bg_none'
proccessed_tiff_folder_name = 'bg_au_050_ta_020'

# process 'prefix' is PointN
process_suffix = '_RowNumber0_Depth_Profile0'
isobar_suffix = '-MassCorrected'
denoise_suffix = '-Filtered'

tiffpath = data_path.joinpath(slide_foldernames[0]).joinpath(process_folder_prefix+slide_foldernames[0]+process_folder_suffix)
rawpath = tiffpath.joinpath(raw_tiff_folder_name)

for rawfilename in glob.glob(f'{str(rawpath.absolute())}/*{process_suffix}.tiff'):
    rp = str(rawpath.joinpath(rawfilename))

raw_im = tiff.read(rp)

#loaded_ims = []
for m in mass_channels:
    # add gamma correction &/or contrast filter here?
    
    global options
    options.append(m)
    
    # convert to float mid-appending
    loaded_ims.append(raw_im[m]/256.)

im_list = Select(layout=Layout(width='auto', height='auto'),
                 options=options,
                 value=options[idx]
                )


im_list.observe(change_image, names='index')


# callback for up-button operation
def move_up(b):
    global idx
    if not idx==0:
        buf_idx = idx
        loaded_ims[idx-1], loaded_ims[idx] = loaded_ims[idx], loaded_ims[idx-1]
        
        global options
        options[idx-1], options[idx] = options[idx], options[idx-1]
        
        im_list.options = options

        idx = buf_idx - 1

        im_list.value = options[idx]
    return

# callback for down-button operation
def move_down(b):
    global idx
    if not idx == len(loaded_ims)-1:
        buf_idx = idx
        loaded_ims[idx+1], loaded_ims[idx] = loaded_ims[idx], loaded_ims[idx+1]
        
        global options
        options[idx+1], options[idx] = options[idx], options[idx+1]
        
        im_list.options = options
        
        idx = buf_idx + 1
        
        im_list.value = options[idx]

b_up.on_click(move_up)
b_down.on_click(move_down)

        
#im_list = Button(layout=Layout(width='auto', height='auto'), 

/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "fov_id" attribute is required if "folder" is specified. Setting "fov_id" to Point1.
  f'The "{field_name}" attribute is required if "{value_name}" '
/usr/local/anaconda3/envs/mibisualization/lib/python3.7/site-packages/mibidata/mibi_image.py:423: UserWarning: The "folder" attribute is required if "fov_id" is specified. Setting "folder" to Point1.
  f'The "{field_name}" attribute is required if "{value_name}" '


In [7]:

grid = GridspecLayout(12,6)

# Image View
grid[:,1:] = im_screen

# Button/Control Panel
grid[0,0] = buttons

# File List
grid[1:,0] = im_list

# wonder if I can get this to work right :/
#grid.layout.border = 'solid 2px'


def manage_mode(b):
    im_list.disabled = True
    for control in buttons.children:
        control.disabled = True
    grid[:,1:] = file_select

b_manage.on_click(manage_mode)
    
display(grid)

with im_screen:
    plt.style.use('dark_background')
    
    ax = plt.gcf().add_subplot(111)
    pltimage = ax.imshow(loaded_ims[0],cmap=cm.afmhot)
    style_plot()
    
    #for im in loaded_ims:
    #    fig = plt.gcf()
    #    ax = fig.add_subplot(111)
    #    pltimage = ax.imshow(im,cmap=cm.gray)
    #    pltimage.set_visible(False)
    #    style_plot()
    #    loaded_ax.append(pltimage)
    
    change_image({'new': idx, 'old': idx})



GridspecLayout(children=(Output(layout=Layout(grid_area='widget001', height='1200px', width='1200px')), HBox(c…